In [1]:
from gurobipy import *

In [2]:
model = Model()

Using license file /home/jamerrq/gurobi.lic
Academic license - for non-commercial use only


## Data

In [3]:
# Number of teams
n = 10

# Number of rounds
k = 18

# Set of teams
I = [i for i in range(n)]

# Set of rounds
K = [i for i in range(k)]

# Set of best teams (Brazil and Argentina)
Is = [0, 2]

# Set of names
# names = ['BRA', 'ARG', 'PER', 'COL', 'URU', 'BOL', 'VEN', 'ECU', 'PAR', 'CHI']
names = ['ARG', 'BOL', 'BRA', 'CHI', 'COL', 'ECU', 'PAR', 'PER', 'URU', 'VEN']

## Variables

In [4]:
# X_ijk = 1 if team i plays against team j in date k

X = []
for i in range(n):
    team = []
    for j in range(n):
        dates = [None] * k
        for kp in range(k):
            dates[kp] = model.addVar(vtype=GRB.BINARY, name='{} vs {} in date {}'.format(names[i], names[j], kp))
        team.append(dates)
    X.append(team)

## Constraints

### Double robin

In [5]:
# 1. Each team plays against other twice
model.addConstrs(quicksum(X[i][j][kp] + X[j][i][kp] for kp in K if kp < n-1) == 1 for i in I for j in I if i != j);

model.addConstrs(quicksum(X[i][j][kp] + X[j][i][kp] for kp in K if kp >= n-1) == 1 for i in I for j in I if i != j);

# 2. Each team plays once as local against the others
model.addConstrs(quicksum(X[i][j][kp] for kp in K) == 1 for i in I for j in I if i != j);

### Compactness

In [6]:
# 3. Each team plays one match once per date
model.addConstrs(quicksum((X[i][j][kp] + X[j][i][kp]) for i in I if i != j) == 1 for j in I for kp in K);

### Top teams

In [7]:
# 4. Top teams
model.addConstrs(quicksum(X[i][j][k] + X[j][i][k] + X[i][j][k + 1] + X[j][i][k + 1] for j in Is) <= 1
                 for i in I if Is.count(i) == 0 for k in K if k < len(K) - 1);

### Balance

In [8]:
# Let's introduce the auxiliar variable y_ik
# which is equal to 1 when the team i has a H-A sequence in the double rounds
Y = []
for i in range(n):
    y_i = [None] * k
    for kp in range(k):
        y_i[kp] = model.addVar(vtype=GRB.BINARY)
    Y.append(y_i)

# 5. Boundaries for y_ik
model.addConstrs(quicksum(Y[i][k] for k in K if k % 2 == 0) >= n//2 - 1 for i in I);

model.addConstrs(quicksum(Y[i][k] for k in K if k % 2 == 0) <= n//2 for i in I);

# 6.
model.addConstrs(quicksum(X[i][j][k] + X[j][i][k + 1] for j in I if j != i) <= 1 + Y[i][k] for i in I for k in K if k % 2 == 0);

# 7.
model.addConstrs(Y[i][k] <= quicksum(X[i][j][k] for j in I if j != i) for i in I for k in K if k % 2 == 0);

# 8.
model.addConstrs(Y[i][k] <= quicksum(X[j][i][k + 1] for j in I if j != i) for i in I for k in K if k % 2 == 0);

## Objective Function

In [9]:
# Let's introduce the variable w_ik
# which is 1 when the team i has an away break in the double round starting in the round k
W = []
for i in range(n):
    w_i = [None] * k
    for kp in range(k):
        w_i[kp] = model.addVar(vtype=GRB.BINARY)
    W.append(w_i)

# 9.
model.addConstrs(quicksum(X[j][i][k] + X[j][i][k + 1] for j in I if j != i) <= 1 + W[i][k] for i in I for k in K if k % 2 == 0);

# 10. Boundaries for w_ik
model.addConstrs(W[i][k] <= quicksum(X[j][i][k] for j in I if j != i) for i in I for k in K if k % 2 == 0);

model.addConstrs(W[i][k] <= quicksum(X[j][i][k + 1] for j in I if j != i) for i in I for k in K if k % 2 == 0);

# Minimize the w_ik

obj = quicksum(W[i][k] for i in I for k in K if k % 2 == 0);

model.setParam(GRB.Param.TimeLimit, 60*5)

model.setObjective(obj, GRB.MINIMIZE)
#model.setObjective(quicksum(X[i][j][kp] for i in I for j in I for kp in K), GRB.MINIMIZE)

Changed value of parameter TimeLimit to 300.0
   Prev: inf  Min: 0.0  Max: inf  Default: inf


## Schemes

### Mirrored

In [10]:
#model.addConstrs(X[i][j][kp] == X[j][i][kp + n - 1] for i in I for j in I if i != j for kp in K if kp < n - 1);

### French

In [11]:
model.addConstrs(X[i][j][0] == X[j][i][-1] for i in I for j in I if i != j);

model.addConstrs(X[i][j][kp] == X[j][i][kp + n - 2] for i in I for j in I if i != j for kp in K[1:n-1]);

### English

In [12]:
#model.addConstrs(X[i][j][n - 2] == X[j][i][n - 1] for i in I for j in I if i != j);

#model.addConstrs(X[i][j][kp] == X[j][i][kp + n] for i in I for j in I if i != j for kp in K[:n-2]);

### Inverted

In [13]:
#model.addConstrs(X[i][j][kp] == X[j][i][2*n - 1 - kp - 2] for i in I for j in I if i != j for kp in K[:n-2]);

### Back to back

In [14]:
#model.addConstrs(X[i][j][kp] == X[j][i][kp + 1] for i in I for j in I if i != j for kp in K[0:len(K):2]);

### Min-Max separation

In [15]:
c = 6
d = 12

#model.addConstrs(quicksum(X[i][j][k2] + X[j][i][k2] for k2 in K if k2 >= kp and k2 <= kp + c) <= 1 for i in I
#                for j in I if i != j for kp in K if kp <= len(K) - c);

#model.addConstrs(quicksum(X[i][j][k2] for k2 in K if k2 >= 0 and (k2 <= kp + d or k2 <= 2 * (n - 1)) and k2 != kp)
#                 >= X[j][i][kp] for i in I for j in I if i != j for kp in K);

### New restrictions [March 6]

In [16]:
# 11. In half of the matches, one as local and one as visitant against the best teams

model.addConstrs(quicksum(X[i][j][kp] for j in Is for kp in K[:n-1]) == 1 for i in I if Is.count(i) == 0);

### New constraints [March 13]

In [17]:
# 12. Balance H-A and A-H sequences

model.addConstrs(quicksum(Y[i][k2] for k2 in range(kp, kp + 3)) <= 2 for i in I for j in I if i != j
                 for kp in K if kp % 2 == 0 and kp < len(K) - 4);

model.addConstrs(quicksum(Y[i][k2] for k2 in range(kp, kp + 3)) >= 1 for i in I for j in I if i != j
                 for kp in K if kp % 2 == 0 and kp < len(K) - 4);

In [18]:
model.optimize()

Gurobi Optimizer version 9.0.1 build v9.0.1rc0 (linux64)
Optimize a model with 3224 rows, 2160 columns and 21932 nonzeros
Model fingerprint: 0x7c0fd579
Variable types: 0 continuous, 2160 integer (2160 binary)
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  Objective range  [1e+00, 1e+00]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 5e+00]
Presolve removed 2317 rows and 1100 columns
Presolve time: 0.05s
Presolved: 907 rows, 1060 columns, 10738 nonzeros
Variable types: 0 continuous, 1060 integer (1060 binary)

Root relaxation: objective 0.000000e+00, 961 iterations, 0.08 seconds

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestBd   Gap | It/Node Time

*    0     0               0       0.0000000    0.00000  0.00%     -    0s

Explored 0 nodes (2928 simplex iterations) in 0.32 seconds
Thread count was 4 (of 4 available processors)

Solution count 1: 0 

Optimal solution found (toleran

In [19]:
row = ' TEM | DT01  DT02  DT03  DT04  DT05  DT06  DT07  DT08  DT09 |'

for i in range(10, 19):
    row += ' DT{} '.format(i)

print(row, '-' * 114, sep='\n')

for i in range(n):
    row = ' {} |'.format(names[i])
    for kp in range(k):
        for j in range(n):
            if X[i][j][kp].X == 1:
                #row += '  {} '.format(names[j])
                row += '  -H- '
            if X[j][i][kp].X == 1:
                #row += ' @{} '.format(names[j])
                row += '  -A- '
        if kp == 8:
            row += '|'
    print(row)

 TEM | DT01  DT02  DT03  DT04  DT05  DT06  DT07  DT08  DT09 | DT10  DT11  DT12  DT13  DT14  DT15  DT16  DT17  DT18 
------------------------------------------------------------------------------------------------------------------
 ARG |  -A-   -H-   -H-   -A-   -A-   -H-   -H-   -A-   -H- |  -A-   -A-   -H-   -H-   -A-   -A-   -H-   -A-   -H- 
 BOL |  -A-   -H-   -H-   -A-   -H-   -A-   -A-   -H-   -H- |  -A-   -A-   -H-   -A-   -H-   -H-   -A-   -A-   -H- 
 BRA |  -H-   -A-   -H-   -A-   -A-   -H-   -A-   -H-   -A- |  -H-   -A-   -H-   -H-   -A-   -H-   -A-   -H-   -A- 
 CHI |  -A-   -H-   -A-   -H-   -H-   -A-   -A-   -H-   -H- |  -A-   -H-   -A-   -A-   -H-   -H-   -A-   -A-   -H- 
 COL |  -A-   -H-   -H-   -A-   -A-   -H-   -A-   -H-   -H- |  -A-   -A-   -H-   -H-   -A-   -H-   -A-   -A-   -H- 
 ECU |  -H-   -A-   -A-   -H-   -H-   -A-   -H-   -A-   -A- |  -H-   -H-   -A-   -A-   -H-   -A-   -H-   -H-   -A- 
 PAR |  -H-   -A-   -A-   -H-   -H-   -A-   -A-   -H-   -A- |  -H-   -H- 